In [9]:
import torch
from noising_functions import gBAOAB_integrator
import torch.nn as nn

In [10]:
bones2 = [
    (1, 2),
    (1, 3),
    (3, 4),
    (4, 5),
    (5, 6),
    (1, 7),
    (7, 8),
    (8, 9),
    (9, 10),
    (11, 12),
    (12, 13),
    (13, 14),
    (15, 16),
    (16, 17),
    (17, 18),
]

In [11]:
def length_constraint(i,j, xinit):
    init = xinit 
    def constraint_fn(y):
        x = torch.squeeze(y) # will need to change for batched data
        return (x[:,3*i]- x[:,3*j])**2 +(x[:,3*i+1]- x[:,3*j+1])**2 + (x[:,3*i+2]- x[:,3*j+2])**2 - ((init[:,3*i]- init[:,3*j])**2 +(init[:,3*i+1]- init[:,3*j+1])**2 + (init[:,3*i+2]- init[:,3*j+2])**2)
    return constraint_fn

In [12]:
def length_constraint_2(i, xinit):
    init = torch.squeeze(xinit)
    def constraint_fn(y):
        x = torch.squeeze(y)
        return (x[:,3*i]- 0)**2 +(x[:,3*i+1]- 0)**2 + (x[:,3*i+2]- 2)**2 - ((init[:,3*i]- 0)**2 +(init[:,3*i+1]- 0)**2 + (init[:,3*i+2]- 2)**2)
    return constraint_fn

In [11]:
def cotangent_projection(gs):
    def proj(x):
        G = J(gs,x)
        M = torch.eye(G.size()[1])
        L= torch.eye(G.size()[1]) - G.t(-2,-1) @ torch.inverse(G @ M @ G.t(-2,-1)) @ G @ torch.inverse(M)
        return L, G
    return proj

In [12]:
class GaussianFourierProjection(nn.Module):
  """Gaussian random features for encoding time steps."""
  def __init__(self, embed_dim, scale=30.):
    super().__init__()
    # Randomly sample weights during initialization. These weights are fixed
    # during optimization and are not trainable.
    self.W = nn.Parameter(torch.randn(embed_dim // 2) * scale, requires_grad=False)

  def forward(self, x):
    x_proj = x[:, None] * self.W[None, :] * 2 * torch.pi
    return torch.cat([torch.sin(x_proj), torch.cos(x_proj)], dim=-1)

In [13]:
class ScoreNet(nn.Module):
  """A time-dependent score-based model."""


  def __init__(self, embed_dim):
    super().__init__()
    self.embed = nn.Sequential(GaussianFourierProjection(embed_dim=embed_dim),nn.Linear(embed_dim, embed_dim))
    self.lin_embed = nn.Linear(embed_dim,57)
    self.lin_embed2 = nn.Linear(embed_dim,57)
    self.lin1 = nn.Linear(57,57)
    self.lin2 = nn.Linear(57, 57)
    self.lin3 = nn.Linear(57, 57)
    self.lin4 = nn.Linear(57, 57)
    self.lin5 = nn.Linear(57,57)
    self.act = torch.nn.Sigmoid()
    
  #@torch.compile(mode="default")
  def forward(self,x,t,L,G):
      # setting the fixed points of x
      l = torch.zeros_like(x)
      l[:,0] = x[:,0]
      l[:,1] = x[:,1]
      l[:,2] = -torch.ones_like(x[:,2])*2 + x[:,2]

      x = x - l
      embed = self.act(self.embed(t))
      h = self.lin1(x)
      h = h+ self.lin_embed(embed)
      h = self.act(self.lin2(h))
      h = self.act(self.lin3(h))+ self.lin_embed2(embed)
      h = self.act(self.lin4(h))
      h = self.lin5(h)

      # projection
      p = L@ h
      h = p
    

      # setting the force on the fixed point to zero
      l2 = torch.zeros_like(h)
      l2[:,0] = h[:,0]
      l2[:,1] = h[:,1]
      l2[:,2] = h[:,2]
      h = h - l2

      return h

In [14]:
score_model = torch.nn.DataParallel(ScoreNet(58))

In [13]:
from torch.utils.data import TensorDataset, DataLoader

In [14]:
from torch.optim import Adam

In [15]:
import warnings
warnings.filterwarnings("ignore")


In [17]:
#torch._dynamo.config.verbose=True

In [18]:
#torch._dynamo.config.suppress_errors = True

# Sampling from the reverse SDE

In [20]:
# using one repeated datapoint as a sanity check
data_tensor = torch.randn(1,57).broadcast_to(1000,57)
my_dataset = TensorDataset(data_tensor)

In [21]:
#@title Define the loss function (double click to expand or collapse)
def loss2(model, xs,eps=1e-5):
  """The loss function for training score-based generative models.

  Args:
    model: A PyTorch model instance that represents a
      time-dependent score-based model.
    x: A mini-batch of training data.
    marginal_prob_std: A function that gives the standard deviation of
      the perturbation kernel.
    eps: A tolerance value for numerical stability.
  """
  # uniformly selecting a time
  loss =torch.tensor([0.])
  loss.requires_grad_()
  tr =0
  s_tr = 0

  # x is the batch of simulated qs that we need to noise.
  gs = [length_constraint(i,j,q) for (i,j) in bones2]
  gs.append(length_constraint_2(1,q))
  gs.append(length_constraint_2(11,q))
  gs.append(length_constraint_2(15,q))

  # projection matrix
  L_fn = cotangent_projection(gs)
  random_t = torch.round(torch.rand(1, device=x.device)*(len(x))-1)

  # Noising
  p = torch.zeros_like(q)
  M = torch.eye(len(q))
  h = 0.01

  sim_x , _ = gBAOAB_integrator(q,p,force, gs, h,M, 1, 1, int(random_t.item()),1,10**(-13))


  L , G= L_fn(sim_x) # defining the projection matrix
  score = model(sim_x, random_t/100,L,G).cpu()

  # need to think about how to let this part act on batches
  tr = tr + torch.abs( torch.trace(torch.squeeze(torch.autograd.functional.jacobian(model, inputs=(sim_x,random_t/100,L,G))[0])) )
  s_tr = s_tr + torch.trace(torch.squeeze(torch.autograd.functional.jacobian(model, inputs=(sim_x,random_t/100,L,G))[0]))
  loss = loss + 1/2 * torch.linalg.norm(score)**2 + torch.trace(torch.squeeze(torch.autograd.functional.jacobian(model, inputs=(sim_x,random_t/100,L,G), create_graph=True,strict=False)[0]))
  
  if tr < 0.1:
    print(f"sum absolute trace = {tr}, sum trace = {s_tr}")
  return loss 

In [ ]:
## size of a mini-batch
## learning rate
lr=1e-3 #@param {'type':'number'}
batch_size =  20 #@param {'type':'integer'}
## learning rate
dataloader = DataLoader(my_dataset,batch_size=batch_size, shuffle=True)
n_epochs = 20
tqdm_epoch = tqdm.notebook.trange(n_epochs)
optimizer = Adam(score_model.parameters(), lr=lr)
i = 0
epoch_losses =[]
for epoch in tqdm_epoch:
    t_dl =tqdm.tqdm(dataloader)
    avg_loss = 0.
    num_items = 0
    for pw in t_dl:
        x = pw[0].cpu()
        i += 1
        loss = loss2(score_model, x)
        t_dl.set_description(f"Loss = {loss.item()}")
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_loss += loss.item() * x.shape[0]
        num_items += x.shape[0]
        epoch_losses.append(avg_loss / num_items)
    tqdm_epoch.set_description('Average Loss: {:5f}'.format(avg_loss / num_items))
    torch.save(score_model.state_dict(), f"ckpt_improved_sig_3_epoch{epoch}.pth")

  0%|          | 0/20 [00:00<?, ?it/s]


  0%|                                                               | 0/250 [00:00<?, ?it/s]No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'

Loss = 1.6969762248829485:   0%|                                    | 0/250 [00:06<?, ?it/s]

sum absolute trace = 0.009552652679137325, sum trace = 0.009552652679137325



Loss = 1.394326041584087:   0%|                             | 1/250 [00:10<27:55,  6.73s/it]

sum absolute trace = 0.006118226521412266, sum trace = 0.0015278494348590493


...

sum absolute trace = 0.08798737943103781, sum trace = -0.08798737943103781



Loss = 0.0789442387024926:   5%|█▎                         | 12/250 [01:00<18:38,  4.70s/it]

sum absolute trace = 0.09675604606442412, sum trace = -0.09675604606442412



Loss = -0.05826776724206838:  22%|█████▌                   | 55/250 [04:16<15:31,  4.78s/it]


Loss = -0.24266382538959025:  30%|███████▌                 | 76/250 [05:50<12:11,  4.20s/it]

In [ ]:
plt.plot(epoch_losses)

In [ ]:
## size of a mini-batch
## learning rate
lr=1e-45 #@param {'type':'number'}
batch_size =  20 #@param {'type':'integer'}
## learning rate
dataloader = DataLoader(my_dataset,batch_size=batch_size, shuffle=True)
n_epochs = 20
tqdm_epoch = tqdm.notebook.trange(n_epochs)
optimizer = Adam(score_model.parameters(), lr=lr)
for epoch in tqdm_epoch:
    t_dl =tqdm.tqdm(dataloader)
    avg_loss = 0.
    num_items = 0
    for pw in t_dl:
        x = pw[0].cpu()
        i += 1
        loss = loss2(score_model, x)
        t_dl.set_description(f"Loss = {loss.item()}")
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_loss += loss.item() * x.shape[0]
        num_items += x.shape[0]
        epoch_losses.append(avg_loss / num_items)
    tqdm_epoch.set_description('Average Loss: {:5f}'.format(avg_loss / num_items))
    torch.save(score_model.state_dict(), 'ckpt_improved_sig_3.pth')

In [ ]:
model = ScoreNet(58)

In [ ]:
x = torch.rand(57)
x.requires_grad_()

# generating a uniform position

In [ ]:
data = np.load('train_dataset.npy')

In [ ]:
bones = [
    (0, 1),
    (1, 2),
    (1, 3),
    (3, 4),
    (4, 5),
    (5, 6),
    (1, 7),
    (7, 8),
    (8, 9),
    (9, 10),
    (0, 11),
    (11, 12),
    (12, 13),
    (13, 14),
    (0, 15),
    (15, 16),
    (16, 17),
    (17, 18),
]

In [ ]:
def angles_to_joints(angles, lengths,bones):
    pose = torch.zeros((19,3))
    pose[0] = torch.tensor([0,0,2])
    for i in range(len(bones)):
        bone = bones[i]
        r = lengths[i]
        phi = torch.tensor(angles[i][0])
        theta = torch.tensor(angles[i][1])
        pose[bone[1]] = pose[bone[0]] + torch.tensor([r*torch.sin(theta)*torch.cos(phi),r*torch.sin(theta)*torch.sin(phi),r*torch.cos(theta)])
    return pose

In [ ]:
def lengths(bones,poses):
    lengths = []
    poses = torch.squeeze(poses).reshape((19,3))
    for bone in bones:
        l = poses[bone[0]] -poses[bone[1]]
        l = torch.sqrt(torch.dot(l,l))
        lengths.append(l)
    return lengths

In [ ]:
def uniform_generator(x_init,bones):
    random_angles = torch.vstack([torch.rand(18)*torch.pi*2,torch.acos(2*torch.rand(18) -1)]).T
    lengths1 = lengths(bones, x_init)
    return angles_to_joints(random_angles,lengths1,bones)

since the original SDE was just Brownian motion, the reverse is simply:

In [ ]:
data = np.load('train_dataset.npy')
x_init = torch.squeeze(torch.tensor(data[0]))

In [ ]:
#ckpt = torch.load('ckpt.pth')
score_model.load_state_dict(ckpt)

In [ ]:
q_init = uniform_generator(torch.squeeze(x_init),bones)

In [ ]:
torch._dynamo.config.suppress_errors = True

In [ ]:
@torch.compile(mode = "max-autotune")
def run_simulation(q_init,steps):
  h = 1/steps
  positions = []
  q = torch.flatten(torch.squeeze(q_init))
  gs = [length_constraint(i,j,torch.squeeze(q)) for (i,j) in bones2]
  gs.append(length_constraint_2(1,torch.squeeze(q)))
  gs.append(length_constraint_2(11,torch.squeeze(q)))
  gs.append(length_constraint_2(15,torch.squeeze(q)))
  p = torch.zeros_like(q)
  M = torch.eye(q.size()[0])
  L_fn = cotangent_projection(gs)
  @torch.compile()
  def force(y):
    with torch.no_grad():
      x = torch.unsqueeze(y,0)
      L = L_fn(x)
      score = -score_model(x,time,L)
    return torch.squeeze(score)
  for step in range(1,steps):
      print(step)
      time = torch.tensor([1-(step)*h])
      q1 = q
      q, p = gBAOAB_step_exact(q,p,force, gs, h,M, 1,1,1,10**(-11))
      print(f"Movement of q: {torch.linalg.norm(q-q1)}")
      positions.append(q)
  return positions

In [ ]:
positions = run_simulation(q_init, 10000)

In [ ]:
pos = np.array([p.detach() for p in positions])
np.save('reverse_pos_new.npy',pos)